# Importar Librerias

In [ ]:
import cv2 #pip install opencv-python
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp #pip install mediapipe
import tensorflow as tf

# Obtencion de "Landmarks" con MediaPipe

In [ ]:
mp_holistic = mp.solutions.holistic # Modelo de MediaPipe (Holistic model)
mp_drawing = mp.solutions.drawing_utils # Utilidades para dibujar

In [ ]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Transformar imagen en BGR desde OpenCV a RGB para MediaPipe
    image.flags.writeable = False                  
    results = model.process(image)                 # Predecir "landmarks" con modelo
    image.flags.writeable = True                   
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # Transformar de vuelta a BGR
    return image, results

In [ ]:
def draw_styled_landmarks(image, results):
    # Dibujar landmarks y conexiones sobre image
    # Para cara
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Para Pose
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Para mano izquierda
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Para mano derecha
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

# Extracción/Creación de vector de features

In [ ]:
def extract_keypoints(results):
    # Results contiene 4 grupos de "landmarks": Face, Pose, Left Hand y Right Hand
    # Cada landmark contiene x,y,z y las poses ademas visibility
    # Face contiene 468 landmarks
    # Pose contiene 33 landmarks
    # Right hand contiene 21 landmarks
    # Left hand contiene 21 landmarks
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

# Creación de carpetas para recolección de data

In [ ]:
# Path para data
DATA_PATH = os.path.join('MP_Data') 

#Acciones/Gestos
actions = np.array(['Counting', 'Waving', 'Bye'])

# Cantidad de videos por gesto
no_sequences = 30

# Numero de frames por video
sequence_length = 40

In [ ]:
# Crear una carpeta por accion/gesto
# Dentro de cada acción no_sequences carpetas, uno para cada video
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# Recolección de data

In [ ]:
cap = cv2.VideoCapture(0)
# Declarar modelo MediaPipe
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # Por cada accion
    for action in actions:
        # Por cada secuencia/video
        for sequence in range(no_sequences):
            # Por cada frame en video
            for frame_num in range(sequence_length):

                # Leer frame
                ret, frame = cap.read()

                # Obtener "landmarks" con MediaPipe
                image, results = mediapipe_detection(frame, holistic)

                # Dibujar "landmarks"
                draw_styled_landmarks(image, results)
                
                # Lógica para hacer break entre cada video
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.imshow('OpenCV Feed', image)
                
                # Obtener features a partir de "landmarks" del frame
                keypoints = extract_keypoints(results)
                # Path para guardar features
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                # Guardarmunpy array
                np.save(npy_path, keypoints)

                # Break grabación presionando "q"
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
    
    #Cerrar camara                
    cap.release()
    cv2.destroyAllWindows()

# Creación de Dataset

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
# Mapa de acciones/gestos
label_map = {label:num for num, label in enumerate(actions)}

In [ ]:
# Se extrajo todos los landmarks para problema general, pero se necesitan solo landmarks de mano derecha aquí
def extract_right_hand(sequence):
    return sequence[-21*3:]

In [ ]:
sequences, labels = [], []
#Por cada accion
for action in actions:
    # Por cada video
    for sequence in range(no_sequences):
        # Array para video actual
        window = []
        # Por cada frame
        for frame_num in range(sequence_length):
            # Cargar array numpy guardado 
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            # Agregar a array de video (extracción de landmarks de mano derecha)
            window.append(extract_right_hand(res))
        # Agregar array de video a array general (Se juntan todos los videos)
        sequences.append(window)
        # Agregar label a array de labels
        labels.append(label_map[action])

In [ ]:
# Pasar python array a numpy array
X = np.array(sequences)

In [ ]:
# Encode labels en formato One-Hot
y = to_categorical(labels).astype(int)

In [ ]:
# Separar en conjunto de entrenamiento y de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

# Creación y entrenamiento de red LSTM

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
#Callback para ver entrenamiento en dashboard de TensorBoard
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
# Definición de modelo con 3 redes LSTM y dos capas fully connected
# Salida on activación softmax para clasificación
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(40,21*3)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [ ]:
# Optimizador Adam, función de loss Categorical Crossentropy
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
# Entrenar el modelo
model.fit(X_train, y_train, epochs=200, callbacks=[tb_callback])

In [ ]:
model.summary()

In [ ]:
#model.save('detector_gestos.h5')
model.load_weights('detector_gestos.h5')

# Cálculo de Accuracy en conjunto de Test

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
yhat = model.predict(X_test)

In [ ]:
# Uso de argmax para extraer label entre los tres gestos (y_test está en One-Hot y yhat está como salida de softmax)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [ ]:
accuracy_score(ytrue, yhat)

# Implementación en tiempo real

In [ ]:
# Funcion para dibujar barras de probabilidad de detección entre gestos, se actualiza frame a frame
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    # res es la salida sofmax del modelo
    output_frame = input_frame.copy()
    # Itera por accion/gesto
    for num, prob in enumerate(res):
        # Dibujar barra sobre image
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        # Escribir label sobre barra
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [ ]:
sequence = [] # Array de features frame a frame
sentence = [] # Array de detecciones validadas (Filtro sobre array predictions)
predictions = [] # Array de detecciones frame a frame
threshold = 0.8 # Umbral de detección

stability_frames = 15 #15 frames con mismo gesto para validar

#Abrir camara
cap = cv2.VideoCapture(0)
# Declarar modelo MediaPipe 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Obtener frame
        ret, frame = cap.read()

        # Obtener "landmarks" desde frame
        image, results = mediapipe_detection(frame, holistic)
                
        # Dibujar landmarks en imagen
        draw_styled_landmarks(image, results)
        
        # Extracción de vector de features
        keypoints = extract_keypoints(results)
        # Extracción solo de landmarks de mano derecha
        keypoints = extract_right_hand(keypoints)
        sequence.append(keypoints)
        # Recortar array sequence para que calce con entrada de modelo
        sequence = sequence[-sequence_length:]
        
        if len(sequence) == sequence_length:
            # Realizar predicción con modelo
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            # Agregar gesto detectado a array predictions
            predictions.append(np.argmax(res))
                        
            
            # Si los últimos stability_frames detectan mismo gesto
            if np.unique(predictions[-stability_frames:])[0]==np.argmax(res): 
                # Si probabilidad supera umbral
                if res[np.argmax(res)] > threshold: 
                    if len(sentence) > 0: 
                        # Si gesto es distinto al último validado
                        if actions[np.argmax(res)] != sentence[-1]:
                            # Agregar a array sentence de gestos validados
                            sentence.append(actions[np.argmax(res)])
                    else:
                        # Caso primer gesto
                        sentence.append(actions[np.argmax(res)])

            # Mostrar solo último 5 gestos validados en pantalla           
            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Mostar barras de probbilidades actual
            image = prob_viz(res, actions, image, colors)
        
        # Escribir gestos validados en parte superior de la pantalla
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Mostrar imagen modificada con "landmarks", barras de probabilidad y gestos detectos sobre
        cv2.imshow('OpenCV Feed', image)

        # Cerrar apretando "q"
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()